### Datasets

All datasets are initially downloaded from [aicrowd](), and then uploaded to Kaggle to have them available there. This saves a lot of time on reloading the data when opening the notebook or running a new run. 

Import datasets from Kaggle directory of jaflaten 
the datasets are 
- food-testing
- food-validation
- food-training


Optional: Checking if the files are in place

In [ ]:
!ls /kaggle/input/food-training


### Importing dependencies
Importing all the dependencies needed for training the model

In [ ]:
from fastai.vision.all import *
import numpy as np
import pandas as pd
import random
from PIL import Image
import os
import json
from pathlib import Path
from fastai.metrics import Precision, Recall, accuracy
import timm

### Defining paths for datasets
Defining the paths for the training and validation set images.

In [ ]:
#define paths

path_validation = Path("/kaggle/input/food-validation/images/")
path_training = Path("/kaggle/input/food-training/images/")

### Loading the metadata 
The annotations.json file is a structure of information which includes a image_id and a category_id which are important when labelling the data. 

Here I load the metadata annotations.json for both training and validation set and creating a mapping between the category id and the name 


In [ ]:
# Load annotations file
train_annotations_path = Path("/kaggle/input/food-training/annotations.json")
val_annotations_path = Path("/kaggle/input/food-validation/annotations.json")

with open(train_annotations_path) as f:
    train_annotations = json.load(f)
    
with open(val_annotations_path) as f:
    val_annotations = json.load(f)    

# Create a dictionary mapping category IDs to their names
categories = {}
for category in train_annotations["categories"]:
    categories[category["id"]] = category["name"]


The annotations.json data is loaded and added to a Pandas Dataframe for each dataset, one for training and one for validation.


In [ ]:
with open(train_annotations_path) as json_data:
    data = json.load(json_data)
    df_train_annotations = pd.DataFrame(data['annotations'])
    
with open(val_annotations_path) as json_data:
    data = json.load(json_data)
    df_val_annotations = pd.DataFrame(data['annotations'])    

Checking the output from each dataframe to verify it looks reasonable

In [ ]:
df_train_annotations.head(2)

In [ ]:
df_val_annotations.head(2)


### merging datasets
Here the training and validation datasets are merged into one make it easier to use when we will create a ImageDataLoader later.

In [ ]:
df_annotations = pd.concat([df_train_annotations, df_val_annotations])

In [ ]:
df_annotations.head(2)

### Labelling the data

To label each image with the correct label based onthe image id I needed to iterate the annotations file and find the images with the corresponding image id similar to the name of the image file. So this function can take a given image name and return the label that should be set to that image. 

In [ ]:

# Define the get_label function
def get_label(image_filename):
    image_id = int(Path(image_filename).stem.lstrip("0"))
    return categories[df_annotations.loc[df_annotations['image_id'] == image_id]['category_id'].values[0]]
    

Testing the get_lable function to verify that we get labels based on a pre-picked image. 

In [ ]:
get_label("059339.jpg")

In [ ]:
get_label("149022.jpg")

### Modifying the dataframe
Creating a new dataframe containing two columns. One for the image name full path including the filename. For instance ```/kaggle/input/food-training/images/059339.jpg```

And one column to determine if the image is from the test or the validation set by setting the ```is_val``` to either False or True. This will be True for the validation data, and false otherwise.

At the end the two dataframes are merged together to one dataframe. 

In [ ]:
df_train = pd.DataFrame(list(path_training.ls()), columns=["img"])
df_val = pd.DataFrame(list(path_validation.ls()), columns=["img"])

df_train["is_val"] = False
df_val["is_val"] = True
df = pd.concat([df_train, df_val])

Verifying dataframe content with manual inspection

In [ ]:
df.head(2)

In [ ]:
df.tail(2)

### Adding the labels
The last modification to the dataframe is to add another column for the label to each image, this is so the model can know what is correct and wrong when being trained. To add the model we can use the get_label function that was defined earlier and do this for every image in the dataframe

In [ ]:
df['label'] = df['img'].apply(get_label)

In [ ]:
df.head(2)

### Creating a ImageDataLoader

To create the ImageDataLoader we want to create it from the dataframe that was stitched together above. The images are being resized and scaled. The labels are set using the get_label function defined above. 

In [ ]:
dls = ImageDataLoaders.from_df(df, label_col=2, valid_col=1, path="/",
                                item_tfms=Resize(192, method='squish'),
                                batch_tfms=aug_transforms(size=128, min_scale=0.75),
                                label_func=get_label)

In [ ]:
#testing resize crop 
#dls.train = dls.train.new(
#    item_tfms=RandomResizedCrop(128, min_scale=0.5),
#    batch_tfms=aug_transforms(size=128, min_scale=0.75)
#)

Checking out a small sample of the images to see what they look like and that they are labeled


In [ ]:
dls.show_batch(max_n=4)

Set a architecture to use

In [ ]:
architecture = 'convnext_large_in22ft1k'
#architecture = 'convnext_small_in22k'

### Metrics and creating a vision learner

We want to have average precision, average recall and accuracy as our metrics. The AP and AR are to be able to compare the model to the leaderboards at [aicrowd](https://www.aicrowd.com/challenges/food-recognition-benchmark-2022/leaderboards). Accuracy will be a useful metric to easily see how well the model performs on the validation set. 

These are used as parameters to the vision_learner when creating a learner. We also specify the architecture to use, the DataLoaders and that we want to use a pretrained model. The last parameter will significantly help us achieve proper results when leveraging a model that has already been trained on a large dataset. 

In [ ]:
average_precision = Precision(average='macro', pos_label=1)
average_recall = Recall(average='macro', pos_label=1)
learner = vision_learner(dls, architecture, metrics=[accuracy, average_precision, average_recall], pretrained=True).to_fp16()

In [ ]:
#learner = vision_learner(dls, resnet34, metrics=accuracy)
#learner = vision_learner(dls, 'convnext_small_in22k', metrics=[ap, ar]).to_fp16()

### Find learning rate
Find the optimal learning rate for the model and use it when fine tuning

In [ ]:
lr = learner.lr_find(suggest_funcs=(minimum, steep, valley), show_plot=True)
base_lr = (lr.valley + lr.steep)/2
print(base_lr)

In [ ]:
learner.fine_tune(5, base_lr=base_lr)

### Export the model
Exporting the model to a file to be used in a application. 


In [ ]:
learner.export(fname = '/kaggle/working/foodmodel.pkl')

In [ ]:
?learner.export

In [ ]:
!ls /kaggle/working
